In [1]:
# imports
import xml.etree.ElementTree as ET
import pandas as pd
import re

In [2]:
# Load the XML file
tree = ET.parse('data/uniprot/UP000000556_160488.xml')

# define the namespace prefixes:
namespaces = {
    'uniprot': 'http://uniprot.org/uniprot',
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
}

root = tree.getroot()


In [3]:
Tags = ["accession", "name", "protein", "gene", "proteinExistence", "sequence"]
lvl2filter = ["domain", 'component']

In [4]:
# Find all <entry> elements within the 'uniprot' namespace
#need to append _1 _2 to repeated tags...

recordsDict = {}
i = 0

for entry_elements in root.findall('uniprot:entry', namespaces=namespaces):
  elementDict = {} # creates

  elementDict['dataset'] = entry_elements.attrib['dataset']


  for element in entry_elements.findall('*', namespaces=namespaces):
    tag = re.findall("(?<=uniprot\}).+", str(element.tag))

    if tag[0] in Tags:
      entry = element.findall("*", namespaces=namespaces)

      if len(entry) == 0:
        data = element.attrib
        if element.text != None:
           elementDict[tag[0]] = element.text
        if "type" in data:
          elementDict[tag[0]] = data['type']
        if "length" in data:
          print(data["length"])
          elementDict[f"{tag[0]}.length"] = data["length"]

      else:
        for lvl2 in element.findall("*", namespaces=namespaces):
          lvl2tag = re.findall("(?<=uniprot\}).+", str(lvl2.tag))
          data = lvl2.attrib
          if lvl2tag[0] not in lvl2filter:
            if len(lvl2) == 0:
              if lvl2.text != None:
                if "type" in data:
                  elementDict[data['type']] = lvl2.text
                else:
                  elementDict[lvl2tag[0]] = data['type']
            else:
              tagList = []

              for lvl3 in lvl2.findall("*", namespaces=namespaces):
                counter = 0
                lvl3tag = re.findall("(?<=uniprot\}).+", str(lvl3.tag))
                
                newKey = f'{lvl2tag[0]}.{lvl3tag[0]}'
                
                if newKey in elementDict:
                  counter += 1
                  
                newKey = f'{newKey}' if counter == 0 else f'{newKey}_{counter}'
                
                elementDict[newKey] = lvl3.text

                
  recordsDict[i] = elementDict
  i += 1


1032
398
91
301
66
284
692
595
443
500
184
167
456
277
379
472
476
323
143
412
556
89
80
400
124
484
776
474
115
569
171
110
171
265
189
51
819
235
373
751
161
683
200
321
420
296
291
301
426
181
128
226
161
301
143
265
95
354
372
232
332
297
410
334
292
229
495
363
166
152
125
229
464
373
450
162
44
323
563
702
77
1299
351
142
452
226
226
207
321
227
480
1399
71
321
283
98
325
539
54
119
60
143
75
404
64
133
78
263
313
406
442
298
117
110
306
323
161
240
478
857
423
152
460
300
197
257
320
450
298
313
157
215
137
481
217
750
386
705
119
401
447
176
228
295
787
214
423
185
175
315
209
250
116
240
387
261
460
184
218
162
194
546
141
413
159
417
412
276
978
224
196
405
261
717
421
857
250
624
231
73
298
418
93
269
544
763
258
221
236
310
228
86
553
765
555
323
265
210
302
298
57
175
104
84
100
103
97
100
473
178
55
619
308
432
303
127
104
190
266
91
298
62
444
250
200
590
267
682
328
106
237
107
710
384
482
170
308
315
400
153
108
95
467
539
313
243
1042
291
151
920
457
90
183
215
347
42

In [5]:
df = pd.DataFrame.from_dict(recordsDict, orient='index').sort_values(by=['ordered locus'])
print(df.head())

#saving as tsv file 
df.to_csv('output/uniprot.tsv', sep="\t", index=False) 

         dataset accession        name submittedName.fullName primary  \
4203  Swiss-Prot    P31857  PARB_PSEPK                    NaN    parB   
107   Swiss-Prot    P31856  Y002_PSEPK                    NaN     NaN   
1206  Swiss-Prot    P25757  RSMG_PSEPK                    NaN    rsmG   
2199  Swiss-Prot    Q88RW8  MNMG_PSEPK                    NaN    mnmG   
2807  Swiss-Prot    P25755  MNME_PSEPK                    NaN    mnmE   

     ordered locus        proteinExistence  \
4203       PP_0001  inferred from homology   
107        PP_0002               predicted   
1206       PP_0003  inferred from homology   
2199       PP_0004  inferred from homology   
2807       PP_0005  inferred from homology   

                                               sequence sequence.length  \
4203  MAVKKRGLGRGLDALLSGPSVSALEEQAVKIDQKELQHLPVELVQR...             290   
107   MAKVFAIANQKGGVGKTTTCINLAASLAATKRRVLLIDLDPQGNAT...             263   
1206  MSSLVTPQHAEELSTGARQLGVELTAEQHEKLLGYLALLIKWNKAY...    

In [6]:

column_names = list(df.columns.values)
#column_names.index(['ordered locus', 'accession','name','primary','dataset'])
print(column_names)

['dataset', 'accession', 'name', 'submittedName.fullName', 'primary', 'ordered locus', 'proteinExistence', 'sequence', 'sequence.length', 'recommendedName.fullName', 'recommendedName.ecNumber', 'alternativeName.fullName', 'alternativeName.fullName_1', 'alternativeName.shortName', 'alternativeName.shortName_1', 'submittedName.ecNumber', 'recommendedName.shortName', 'synonym', 'recommendedName.shortName_1', 'ORF', 'submittedName.ecNumber_1', 'alternativeName.ecNumber', 'recommendedName.ecNumber_1', 'alternativeName.ecNumber_1']
